In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

! pip install -U scikit-learn

from sklearn.model_selection import train_test_split
from sklearn import model_selection
import plotly.express as px

Defaulting to user installation because normal site-packages is not writeable


In [3]:
data = pd.read_parquet("../data/processed/CellPhoneDs_process.parquet")

data.drop(['wifi', 'blue', 'four_g', 'dual_sim', 'three_g', 'touch_screen'], axis=1, inplace=True)

data["price_range"] = data["price_range"].astype('category')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328 entries, 0 to 2327
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   talk_time      2328 non-null   int64   
 1   battery_power  2328 non-null   int64   
 2   pc             2328 non-null   int64   
 3   mobile_wt      2328 non-null   int64   
 4   px_width       2328 non-null   int64   
 5   sc_h           2328 non-null   int64   
 6   sc_w           2328 non-null   int64   
 7   m_dep          2328 non-null   float64 
 8   fc             2328 non-null   int64   
 9   price_range    2328 non-null   category
 10  n_cores        2328 non-null   int64   
 11  ram            2328 non-null   int64   
 12  int_memory     2328 non-null   int64   
 13  px_height      2328 non-null   int64   
 14  clock_speed    2328 non-null   float64 
dtypes: category(1), float64(2), int64(12)
memory usage: 257.2 KB


In [5]:
# import pandas profilling lib
from pandas_profiling import ProfileReport
profile3 = ProfileReport(data, title="Diabetes data Profiling Report 3" )
profile3.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Product space too large to allocate arrays!

In [5]:
profile3.to_file("../reports/CellPhoneDs_profiling_bivariate_report_3.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
duplicate = data.duplicated(keep='last')
columnDateFrame = data.columns.values
# dupli = []
dupli = pd.DataFrame({},columns = columnDateFrame)
for i in range(len(data)):
    if duplicate.iloc[i] == True:
        # dupli.append(i)
        data.drop(i, inplace=True)
        # dupli = dupli.append(data.iloc[i])
        # dupli=data.iloc[i]    

print(f'{len(dupli)}')
# duplicate

0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1965 entries, 0 to 2327
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   talk_time      1965 non-null   int64   
 1   battery_power  1965 non-null   int64   
 2   pc             1965 non-null   int64   
 3   mobile_wt      1965 non-null   int64   
 4   px_width       1965 non-null   int64   
 5   sc_h           1965 non-null   int64   
 6   sc_w           1965 non-null   int64   
 7   m_dep          1965 non-null   float64 
 8   fc             1965 non-null   int64   
 9   price_range    1965 non-null   category
 10  n_cores        1965 non-null   int64   
 11  ram            1965 non-null   int64   
 12  int_memory     1965 non-null   int64   
 13  px_height      1965 non-null   int64   
 14  clock_speed    1965 non-null   float64 
dtypes: category(1), float64(2), int64(12)
memory usage: 232.4 KB


In [8]:
data.isnull().sum()

talk_time        0
battery_power    0
pc               0
mobile_wt        0
px_width         0
sc_h             0
sc_w             0
m_dep            0
fc               0
price_range      0
n_cores          0
ram              0
int_memory       0
px_height        0
clock_speed      0
dtype: int64

In [9]:
duplicate = data.duplicated()
count=0
for i in range(len(data)):
    if duplicate.iloc[i] == True:
        count=count+1
count

0

In [10]:
# input and output 

X = data.drop("price_range", axis=1)
y= data["price_range"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3,random_state =123, stratify=y)

In [12]:
perc_test = y_test.value_counts()[1]*100/len(y_test)
perc_train = y_train.value_counts()[1]*100/len(y_train)

print(f" Percentage of positives in test set={perc_test} , train set={perc_train}") 

 Percentage of positives in test set=25.084745762711865 , train set=25.09090909090909


In [13]:
# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [14]:
# Selected algorithms
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [15]:
seed =2
models = []

#logistic Regression
models.append(('KNN', KNeighborsClassifier(n_neighbors=4)))

# Decision Tree classifier
models.append(('CART', DecisionTreeClassifier()))

# Naïve Bayes
models.append(('NB', GaussianNB()))
# cluster
models.append(('RF', RandomForestClassifier()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
	# Kfol cross validation for model selection
	kfold = model_selection.KFold(n_splits=10, shuffle=True)
	#X train , y train
	cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring = scoring)
	results.append(cv_results)
	names.append(name)
	msg = f"({name}, {cv_results.mean()}, {cv_results.std()})"
	print(msg)

(KNN, 0.9098275679678409, 0.025635490587551983)
(CART, 0.8429175922987412, 0.038422334977669906)
(NB, 0.8116312281815297, 0.03548198919374836)
(RF, 0.8894689516555591, 0.03167949639308954)


In [16]:
result_df = pd.DataFrame(results, index=names).T
px.box(result_df,title = 'Algorithm Comparison')

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
#priors calculation
y.value_counts()/len(y)

1    0.250891
3    0.250382
0    0.249873
2    0.248855
Name: price_range, dtype: float64

In [19]:
a,b,c,d = y.value_counts()/len(y)

In [20]:
parameters = {'priors': [None, [a,b,c,d]]}

grid_search = GridSearchCV(GaussianNB(), parameters, cv=10, return_train_score=True)
grid_search.fit(X_train, y_train)

grid_search.best_params_

{'priors': None}

In [21]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_precision_recall_curve

In [22]:
RF = RandomForestClassifier()
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       147
           1       0.81      0.81      0.81       148
           2       0.79      0.82      0.81       147
           3       0.94      0.91      0.92       148

    accuracy                           0.87       590
   macro avg       0.87      0.87      0.87       590
weighted avg       0.87      0.87      0.87       590



In [24]:
plot_confusion_matrix(RF, X_test, y_test,cmap=plt.cm.Blues); 

/home/hfrancoo/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.



In [25]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['log2', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [26]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   3.1s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   3.2s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   3.3s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   8.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=  14.9s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time=  14.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000;

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)